# Setup

In [2]:
import os
import pickle
from typing import List, Dict
import uuid

import pandas as pd

import mlflow

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline

In [3]:
# Setup input and output file
year = 2022
month = 2
taxi_type = "green"

input_file = f"https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet"
output_file = f"output/{taxi_type}/{year:04d}-{month:02d}.parquet"

In [4]:
# Setup AWS credential
os.environ["AWS_PROFILE"] = "Profile1"

# Get model from S3
RUN_ID = os.getenv("RUN_ID", "95c848791a7642ff8c26794d43e410a8")
logged_model = f"s3://taxi-mlops/1/{RUN_ID}/artifacts/model"
model = mlflow.pyfunc.load_model(logged_model)
model

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.sklearn
  run_id: 95c848791a7642ff8c26794d43e410a8

# Helper function

In [5]:
def read_dataframe(filename: str) -> pd.DataFrame:
    """Read DataFrame and target column from secs to mins."""
    df = pd.read_parquet(filename)

    df["duration"] = df["lpep_dropoff_datetime"] - df["lpep_pickup_datetime"]
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60.0)
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    df["ride_id"] = generate_uuids(len(df))
    return df

def prepare_dictionaries(df: pd.DataFrame) -> List[Dict]:
    """Turn DataFrame into list of dictionary."""
    # convert data type
    categorical_str = ["PULocationID", "DOLocationID"]
    df[categorical_str] = df[categorical_str].astype(str)

    # Feature engineering
    df["PU_DO"] = df["PULocationID"] + "_" + df["DOLocationID"]
    categorical = ["PU_DO"]
    numerical = ["trip_distance"]
    dicts = df[categorical + numerical].to_dict(orient="records")
    return dicts

def generate_uuids(n):
    return [str(uuid.uuid4()) for _ in range(n)]

# Inference

In [6]:
def load_model(run_id):
    logged_model = f"s3://taxi-mlops/1/{run_id}/artifacts/model"
    model = mlflow.pyfunc.load_model(logged_model)
    return model

def apply_model(input_file, run_id, output_file):
    df = read_dataframe(input_file)
    dicts = prepare_dictionaries(df)

    model = load_model(run_id)
    y_pred = model.predict(dicts)

    df_result = pd.DataFrame()
    df_result["ride_id"] = df["ride_id"]
    df_result["lpep_pickup_datetime"] = df["lpep_pickup_datetime"]
    df_result["PULocationID"] = df["PULocationID"]
    df_result["DOLocationID"] = df["DOLocationID"]
    df_result["actual_duration"] = df["duration"]
    df_result["predicted_duration"] = y_pred
    df_result["diff"] = df_result["actual_duration"] - df_result["predicted_duration"]
    df_result["model_version"] = run_id
    df_result.to_parquet(output_file, index=False)

    return df_result


In [7]:
result = apply_model(input_file, RUN_ID, output_file)

In [12]:
result["diff"].mean()

0.6884502571969741